In [ ]:
import logging
import asyncio

from ezib_async.ezib import ezIBAsync
from ezib_async import util

In [ ]:
util.logToConsole('DEBUG')
logging.getLogger('ib_async').setLevel('CRITICAL')

In [ ]:
# initialize ezIBAsync
ezib = ezIBAsync()

# connect to IB (7496/7497 = TWS, 4001 = IBGateway)
await ezib.connectAsync(ibclient=0, ibport=4001, account="U9860850")
# await ezib.connectAsync(ibport=4002, ibclient=100)

### Create some market contracts

In [ ]:
# create some market contracts using dedicated methods
contracts = await asyncio.gather(
	# market contracts
	# ezib.createStockContract("NVDA"),
	# ezib.createStockContract('PLTR'),
	# ezib.createStockContract('NFLX'),
	# ezib.createStockContract('META'),
	# ezib.createStockContract('TSLA'),
	
	# # option contracts
	# ezib.createOptionContract("AAPL", expiry="20251219", strike=200, otype="P"),
	# ezib.createOptionContract("AAPL", expiry="20251219", strike=200, otype="C"),
	
	# # futures contracts
	ezib.createFuturesContract("ES", expiry="202512", exchange="CME"),
	# ezib.createContract("CL", "FUT", "NYMEX", "USD", "202512", 0.0, ""),
	
	# # forex contracts
	# ezib.createForexContract("EUR", currency="USD"),

	# ...or using a contract tuple
	# ezib.createContract("CL", "FUT", "NYMEX", "USD", "202506", 0.0, "")
)

In [ ]:
# Request all registered contracts
await ezib.requestMarketData()

# Wait for data
await asyncio.sleep(5)

In [ ]:
[nan_col for nan_col in ezib.optionsData[16].columns if ezib.optionsData[16][nan_col].isnull().any()]

In [ ]:
ezib.marketData[9]

In [ ]:
ezib.optionsData[15]

In [ ]:
ezib.optionsData[16][['volume', 'ask_delta', 'bid_imp_vol', 'ask_imp_vol', 'last_imp_vol', 'imp_vol']]

In [ ]:
crwv = ezib.ib.tickers()[0]

In [ ]:
ezib.portfolio

In [ ]:
contract = await ezib.createForexContract('EUR')

In [ ]:
ezib.account

In [ ]:
ezib.cancelMarketData()

In [ ]:
# 1. 输入symbol
# 2. 
pd.DataFrame([ezib.contract_to_tuple(c.contract) for c in ezib.ib.tickers()], 
             columns=['symbol', 'sec_type', 'exchange', 'currency', 'expiry', 'strike', 'opt_type']) \
    .sort_values(by=['symbol', 'sec_type'], ascending=False)

In [ ]:
# request market data for all created contracts
await ezib.requestMarketData()

In [ ]:
util.df(ezib.ib.orders())

In [ ]:
import ib_async


[attr for attr in dir(ib_async.Ticker) if not attr.startswith('_')]

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[ezib.contractString(c) for c in ezib.contracts if c.secType not in ("OPT", "FOP", "FUT")],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'last', 'prevLast', 'lastSize', 'prevLastSize','volume'])

def onMarketTickerUpdate(tickers):
    for t in tickers:
        df.loc[ezib.contractString(t.contract)] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.last, t.prevLast, t.lastSize, t.prevLastSize, t.volume)
        clear_output(wait=True) 
    display(df)
    # print(t)
    
ezib.pendingMarketTickersEvent += onMarketTickerUpdate
await asyncio.sleep(30)
ezib.pendingMarketTickersEvent -= onMarketTickerUpdate

In [ ]:
util.df(ezib.portfolios)

In [ ]:
ezib.portfolio

In [ ]:
ezib.connected

In [ ]:
# cancel market data request & disconnect
ezib.cancelMarketData()


In [ ]:
ezib.disconnect()